<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Generative AI with OpenAI API</h1>
<h1>Embeddings</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from ipywidgets import interact
import tiktoken

import openai
from openai import OpenAI

import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams

import scipy
from scipy.spatial.distance import cosine as cosine_similarity

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm
tqdm.pandas()

import watermark

%load_ext watermark

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: f398f57a1163e8aae0ec0bd88c200edc599ffc4a

nltk     : 3.8.1
scipy    : 1.11.4
tqdm     : 4.66.4
watermark: 2.4.3
openai   : 1.30.5
tiktoken : 0.7.0
numpy    : 1.26.4
pandas   : 2.1.4



Get the API key

In [3]:
client = OpenAI()

# Extract embeddings

In [4]:
response = client.embeddings.create(
    model="text-embedding-ada-002",
    input="Your text goes here",
    )

In [5]:
response

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.013035915791988373, -0.013468892313539982, 0.009552122093737125, -0.02159552462399006, -0.025019366294145584, 0.04396374523639679, -0.01969042792916298, -0.015773657709360123, -0.01076445635408163, -0.023300783708691597, 0.005355583503842354, 0.016599643975496292, -0.009865197353065014, -0.0009741965914145112, 0.0031074373982846737, -0.006441354751586914, 0.024379894137382507, -0.004003365058451891, 0.01446806825697422, -0.012229913845658302, -0.004576226230710745, 0.013302363455295563, 0.013215768150985241, 0.012249897234141827, -0.004729432985186577, 0.001040808274410665, 0.01185688842087984, -0.01723911613225937, 0.015986815094947815, 0.009512155316770077, 0.005918452516198158, -0.006298139225691557, -0.02020999975502491, -0.006880992092192173, -0.017811978235840797, -0.009099163115024567, 0.0024962746538221836, -0.015893559902906418, 0.02787034958600998, -0.00518239289522171, 0.011370622552931309, 0.007733622100204229, -0.001591

The embedding itself is a high dimensional vector

In [6]:
response.data[0].embedding

[-0.013035915791988373,
 -0.013468892313539982,
 0.009552122093737125,
 -0.02159552462399006,
 -0.025019366294145584,
 0.04396374523639679,
 -0.01969042792916298,
 -0.015773657709360123,
 -0.01076445635408163,
 -0.023300783708691597,
 0.005355583503842354,
 0.016599643975496292,
 -0.009865197353065014,
 -0.0009741965914145112,
 0.0031074373982846737,
 -0.006441354751586914,
 0.024379894137382507,
 -0.004003365058451891,
 0.01446806825697422,
 -0.012229913845658302,
 -0.004576226230710745,
 0.013302363455295563,
 0.013215768150985241,
 0.012249897234141827,
 -0.004729432985186577,
 0.001040808274410665,
 0.01185688842087984,
 -0.01723911613225937,
 0.015986815094947815,
 0.009512155316770077,
 0.005918452516198158,
 -0.006298139225691557,
 -0.02020999975502491,
 -0.006880992092192173,
 -0.017811978235840797,
 -0.009099163115024567,
 0.0024962746538221836,
 -0.015893559902906418,
 0.02787034958600998,
 -0.00518239289522171,
 0.011370622552931309,
 0.007733622100204229,
 -0.00159118778537

In [7]:
len(response.data[0].embedding)

1536

And the similarity between vectors represents the similarity between the texts

In [8]:
emb1 = client.embeddings.create(
    model="text-embedding-ada-002",
    input="love",
    ).data[0].embedding

emb2 = client.embeddings.create(
    model="text-embedding-ada-002",
    input="kindness",
    ).data[0].embedding

Cosine similarity is a quick and dirty way to measure how similar two vectors are

In [9]:
cosine_similarity(emb1, emb2)

0.1624071663072032

Naturally, unrelated words have much lower degree of similarity

In [10]:
emb3 = client.embeddings.create(
    model="text-embedding-ada-002",
    input="airplane",
    ).data[0].embedding

In [11]:
cosine_similarity(emb2, emb3)

0.22989575199368795

# Generate embeddings for the Reuters sentences dataset

We're going to use Reuters

In [12]:
sentences = [" ".join(word for word in sent) for sent in reuters.sents()]

In [13]:
len(sentences)

54716

In [14]:
sentences[:10]

["ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said .",
 'They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products .',
 "But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain .",
 "The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost .",
 'Unofficial Japanese estimates put the impact of the tariffs at 10 billion dlrs and spokesmen for major electronics firms said they would virtually halt expo

Let us build a DataFrame of 1000 sentences

In [15]:
data = pd.DataFrame(sentences[:1000], columns=["text"])

In [16]:
data

,text
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...
1,They told Reuter correspondents in Asian capit...
2,But some exporters said that while the conflic...
3,The U . S . Has said it will impose 300 mln dl...
4,Unofficial Japanese estimates put the impact o...
...,...
995,The decision to cut deposits was taken by the ...
996,The cuts were likely to attract more business ...
997,ELECTRO RENT CORP & lt ; ELRC > 3RD QTR FEB 28...
998,WALGREEN CO 2ND QTR SHR 62 CTS VS 58 CTS


In [17]:
embedding_model = "text-embedding-ada-002"

In [18]:
encoding = tiktoken.encoding_for_model(embedding_model)

# Compute the embeddings for each sentence

In [19]:
def get_embedding(text, engine):
    text = text.replace("\n", " ")
    
    return client.embeddings.create(input=[text], model=engine).data[0].embedding

In [20]:
data["embedding"] = data.text.progress_apply(lambda x: get_embedding(x, engine=embedding_model))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [21]:
data

,text,embedding
0,ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPA...,"[-0.016333874315023422, -0.02825174853205681, ..."
1,They told Reuter correspondents in Asian capit...,"[-0.021920351311564445, -0.016266927123069763,..."
2,But some exporters said that while the conflic...,"[-0.020737407729029655, -0.01833837479352951, ..."
3,The U . S . Has said it will impose 300 mln dl...,"[-0.019844893366098404, -0.01669744960963726, ..."
4,Unofficial Japanese estimates put the impact o...,"[-0.02124229073524475, -0.033732276409864426, ..."
...,...,...
995,The decision to cut deposits was taken by the ...,"[0.003799356985837221, -0.014849599450826645, ..."
996,The cuts were likely to attract more business ...,"[-0.020356539636850357, -0.02053343504667282, ..."
997,ELECTRO RENT CORP & lt ; ELRC > 3RD QTR FEB 28...,"[0.0015332361217588186, -0.012259138748049736,..."
998,WALGREEN CO 2ND QTR SHR 62 CTS VS 58 CTS,"[0.002342945197597146, 0.006453890353441238, 0..."


# Generate Recommendations

In [22]:
def get_recommendations(data, pos, k_nearest_neighbors=3):
    embeddings = np.array(data["embedding"].to_list())
    strings = data["text"].to_list()

    query_embedding = embeddings[pos]

    distances = [ cosine_similarity(query_embedding, embedding) for embedding in tqdm(embeddings, total=len(embeddings)) ]
    indices_of_nearest_neighbors = np.argsort(distances)

    query_string = strings[pos]
    print(f"Query: {query_string}\n\n")

    for pos, i in enumerate(indices_of_nearest_neighbors[1:k_nearest_neighbors+1]):
        print("-- Nearest neighbor %u of %u (%u)---\n%s\n\n" % (pos, k_nearest_neighbors, i, strings[i]))

In [23]:
%%time
get_recommendations(data, 123, 5)

  0%|          | 0/1000 [00:00<?, ?it/s]

Query: It will be the next decade before we see if the strategy is right or wrong ."


-- Nearest neighbor 0 of 5 (94)---
But financial analysts are divided on whether and how quickly the gamble will pay off .


-- Nearest neighbor 1 of 5 (118)---
We expect this will be allowed in two or three years ," he said .


-- Nearest neighbor 2 of 5 (106)---
We only have to wait two or three years , not until the 21st century ," Komatsu said .


-- Nearest neighbor 3 of 5 (321)---
Bankers say it is too early to venture a forecast for economic growth this year or next .


-- Nearest neighbor 4 of 5 (580)---
" And we will have to see if the United States is able to do what they promised in Paris on reducing the budget deficit -- and get it through Congress ," he added .


CPU times: user 48.7 ms, sys: 3.47 ms, total: 52.2 ms
Wall time: 51.1 ms


<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>